In [ ]:
import keras
import numpy as np
import time

In [ ]:
model_file = 'timeseries_feedforward_model.json'

# Load the model
with open(model_file, 'r') as json_file:
        model = keras.models.model_from_json(json_file.read())
        
# Compile the model
model.compile(optimizer='adadelta', loss='categorical_crossentropy',
                      metrics=['accuracy'])

In [ ]:
data_file = 'whale_training_samples19200train4800val6000test_shape4000.hdf5'

# Define normalizer for loading data
def normalize_long_axis(x):
    # Assuming data format (num_samples, time_series, channels)
    # and we want to normalize over the time series
    return keras.utils.normalize(x, axis=1)

# Load training and validataion data
# Data is normalized upon loading
training_data = keras.utils.io_utils.HDF5Matrix(
        data_file, 'training_data', normalizer=normalize_long_axis)
validation_data = keras.utils.io_utils.HDF5Matrix(
        data_file, 'validation_data', normalizer=normalize_long_axis)
training_labels = keras.utils.io_utils.HDF5Matrix(
        data_file, 'training_labels')
validation_labels = keras.utils.io_utils.HDF5Matrix(
        data_file, 'validation_labels')

In [ ]:
keras.backend.set_image_data_format('channels_last')

In [ ]:
# Callbacks
callbacks = []

batch_size = 100
epochs = 10
# Save training log
callbacks.append(keras.callbacks.CSVLogger("training_log_"+time.strftime("%Y%m%d")+"_"
                                           +str(batch_size)+"batch_"
                                           +str(epochs)+"epochs_"
                                           +model_file[:-5]+".csv"))

In [ ]:
# Train
model.fit(training_data, training_labels, epochs=epochs,
          batch_size=batch_size, verbose=2, shuffle='batch',
          validation_data=(validation_data, validation_labels),
          callbacks=[])
